In [1]:
!apt-get update
!apt-get install -y build-essential nvidia-cuda-toolkit

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease                          
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease         
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                                      
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease                                    
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease               
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,730 kB]
Get:12 h

## Tạo file mã nguồn CUDA

In [2]:
%%writefile cuda_info.cu
#include <iostream>
#include <cuda_runtime.h>
using namespace std;
int main(int argc, char ** argv)
{
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    for (int dev = 0; dev < deviceCount; dev++)
    {
        cudaDeviceProp deviceProp;
        cudaGetDeviceProperties(&deviceProp, dev);
        if (dev == 0)
        {
            if (deviceProp.major == 9999 && deviceProp.minor == 9999)
            {
                cout << "No CUDA GPU has been detected" << endl;
                return -1;
            }
            else if (deviceCount == 1)
            {
                cout << "There is 1 device supporting CUDA" << endl;
            }
            else
            {
                cout << "There are " << deviceCount << " devices supporting CUDA" << endl;
            }
        }
        cout << "Device " << dev << " name: " << deviceProp.name << endl;
        cout << " Computational Capabilities: " << deviceProp.major << "." << deviceProp.minor << endl;
        cout << " Maximum global memory size: " << deviceProp.totalGlobalMem << endl;
        cout << " Maximum constant memory size: " << deviceProp.totalConstMem << endl;
        cout << " Maximum shared memory size per block: " << deviceProp.sharedMemPerBlock << endl;
        cout << " Maximum block dimensions: " << deviceProp.maxThreadsDim[0] << " x " << deviceProp.maxThreadsDim[1] << " x " << deviceProp.maxThreadsDim[2] << endl;
        cout << " Maximum grid dimensions: " << deviceProp.maxGridSize[0] << " x " << deviceProp.maxGridSize[1] << " x " << deviceProp.maxGridSize[2] << endl;
        cout << " Warp size: " << deviceProp.warpSize << endl;
    }
    cudaDeviceReset();
    return 0;
}

Overwriting cuda_info.cu


## Biên dịch mã CUDA

In [3]:
!nvcc cuda_info.cu -o cuda_info

In [4]:
!./cuda_info

There are 2 devices supporting CUDA
Device 0 name: Tesla T4
 Computational Capabilities: 7.5
 Maximum global memory size: 15828320256
 Maximum constant memory size: 65536
 Maximum shared memory size per block: 49152
 Maximum block dimensions: 1024 x 1024 x 64
 Maximum grid dimensions: 2147483647 x 65535 x 65535
 Warp size: 32
Device 1 name: Tesla T4
 Computational Capabilities: 7.5
 Maximum global memory size: 15828320256
 Maximum constant memory size: 65536
 Maximum shared memory size per block: 49152
 Maximum block dimensions: 1024 x 1024 x 64
 Maximum grid dimensions: 2147483647 x 65535 x 65535
 Warp size: 32


## Merge Sort

### 1. Ý tưởng thuật toán

In [ ]:
from IPython.display import display, Markdown

def print_formatted_output():
    data = """
    |                   Input                   |
    |-------------------------------------------|
    |      8 3 1 9 1 2 7 5 9 3 6 4 2 0 2 5      |
    |                                           |
    |  Thread1 | Thread2  | Thread3  | Thread4  |
    |----------|----------|----------|----------|
    | 8 3 1 9  | 1 2 7 5  | 9 3 6 4  | 2 0 2 5  |
    |  38 19   |  12 57   |  39 46   |  02 25   |
    |---------------------|---------------------|
    |       Thread1       |        Thread2      |
    |   1398       1257   |   3469       0225   |
    |       11235789      |       02234569      |
    |-------------------------------------------|
    |                  Thread1                  |
    |      0 1 1 2 2 2 3 3 4 5 5 6 7 8 9 9      |
    |                                           |
    """
    display(Markdown(f'```\n{data}\n```'))

print_formatted_output()

### 2. Prepare input arrays

In [5]:
import random

def generate_merge_sort_input_file(filename, size):
    """
    Generate an input file for merge sort with a 1D array of random integers.
    - `filename`: Name of the output file.
    - `size`: Number of elements in the array.
    """
    with open(filename, 'w') as f:
        # Write the size of the array
        f.write(f"{size}\n")
        
        # Generate a list of random integers (range: 0 to 9999)
        numbers = [str(random.randint(0, 9999)) for _ in range(size)]
        
        # Write numbers to file
        f.write(" ".join(numbers) + "\n")

# List of sizes to generate input files
sizes = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]

# Generate test cases for all sizes
for size in sizes:
    filename = f"MERGE_SORT_{size}.INP"
    generate_merge_sort_input_file(filename, size)

print("✅ Merge sort input files generated successfully!")

✅ Merge sort input files generated successfully!


## CPU Merge Sort (Cython)

In [6]:
import random
import numpy as np
import cudf
import cupy as cp
import cython
from numba import cuda, int32
from timeit import default_timer as timer
from scipy.sparse import lil_matrix
from pyspark.sql import SparkSession
from pyspark.mllib.linalg.distributed import *
import matplotlib.pyplot as plt

In [7]:
%load_ext cython

In [9]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)  # Disable bounds checking for speed
@cython.wraparound(False)   # Disable negative index wrapping
@cython.cdivision(True)  # Enable C-style division
cdef void _merge(np.ndarray[np.int32_t] arr, int left, int mid, int right):
    cdef int n1 = mid - left + 1
    cdef int n2 = right - mid
    cdef int i, j, k

    # Allocate memory for temporary arrays
    cdef np.ndarray[np.int32_t] L = np.empty(n1, dtype=np.int32)
    cdef np.ndarray[np.int32_t] R = np.empty(n2, dtype=np.int32)

    # Copy data into temporary arrays
    for i in range(n1):
        L[i] = arr[left + i]
    for j in range(n2):
        R[j] = arr[mid + 1 + j]

    # Merge temporary arrays back into arr[left:right+1]
    i = 0
    j = 0
    k = left
    while i < n1 and j < n2:
        if L[i] <= R[j]:
            arr[k] = L[i]
            i += 1
        else:
            arr[k] = R[j]
            j += 1
        k += 1

    # Copy remaining elements of L[], if any
    while i < n1:
        arr[k] = L[i]
        i += 1
        k += 1

    # Copy remaining elements of R[], if any
    while j < n2:
        arr[k] = R[j]
        j += 1
        k += 1

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void _merge_sort(np.ndarray[np.int32_t] arr, int left, int right):
    cdef int mid
    if left < right:
        mid = left + (right - left) // 2  # Compute middle index
        _merge_sort(arr, left, mid)      # Recursively sort left half
        _merge_sort(arr, mid + 1, right) # Recursively sort right half
        _merge(arr, left, mid, right)    # Merge sorted halves

def cpu_merge_sort(np.ndarray[np.int32_t] arr):
    """
    Sorts a NumPy array of integers using Merge Sort with Cython optimizations.
    """
    cdef int n = arr.shape[0]
    _merge_sort(arr, 0, n - 1)  # Call optimized merge sort
    return arr

In [10]:
import numpy as np
import time

def read_input_file(filename):
    """
    Reads an input file and returns the array of integers.
    """
    with open(filename, 'r') as f:
        size = int(f.readline().strip())  # Read size (not used in NumPy)
        numbers = np.array(list(map(int, f.readline().strip().split())), dtype=np.int32)
    return numbers

def write_output_file(filename, sorted_numbers):
    """
    Writes the sorted array to an output file.
    """
    with open(filename, 'w') as f:
        f.write(" ".join(map(str, sorted_numbers)) + "\n")

# List of sizes
sizes = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]

# Process each input file
for size in sizes:
    input_filename = f"MERGE_SORT_{size}.INP"
    output_filename = f"MERGE_SORT_{size}.OUT"

    print(f"Processing {input_filename}...")

    # Read input file
    arr = read_input_file(input_filename)

    # Measure sorting time
    start_time = time.time()
    sorted_arr = cpu_merge_sort(arr)
    end_time = time.time()

    # Write output file
    write_output_file(output_filename, sorted_arr)

    print(f"✅ {output_filename} saved! Sorting time: {(end_time - start_time) * 1000:.3f} ms")

print("✅ All files processed successfully!")

Processing MERGE_SORT_100.INP...
✅ MERGE_SORT_100.OUT saved! Sorting time: 0.295 ms
Processing MERGE_SORT_500.INP...
✅ MERGE_SORT_500.OUT saved! Sorting time: 1.144 ms
Processing MERGE_SORT_1000.INP...
✅ MERGE_SORT_1000.OUT saved! Sorting time: 1.911 ms
Processing MERGE_SORT_5000.INP...
✅ MERGE_SORT_5000.OUT saved! Sorting time: 9.478 ms
Processing MERGE_SORT_10000.INP...
✅ MERGE_SORT_10000.OUT saved! Sorting time: 17.058 ms
Processing MERGE_SORT_50000.INP...
✅ MERGE_SORT_50000.OUT saved! Sorting time: 86.552 ms
Processing MERGE_SORT_100000.INP...
✅ MERGE_SORT_100000.OUT saved! Sorting time: 168.884 ms
Processing MERGE_SORT_500000.INP...
✅ MERGE_SORT_500000.OUT saved! Sorting time: 828.747 ms
Processing MERGE_SORT_1000000.INP...
✅ MERGE_SORT_1000000.OUT saved! Sorting time: 1596.694 ms
✅ All files processed successfully!


### 3. CUDA Merge Sort

In [11]:
%%writefile mergeSortCUDA.cu
#include <iostream>
#include <fstream>
#include <cuda_runtime.h>
#include <vector>
#include <string>

#define THREADS 256  // Number of threads per block

using namespace std;

// Device function to merge two sorted subarrays
__device__ void merge(float* data, float* temp, int left, int mid, int right) {
    int i = left, j = mid, k = left;
    
    while (i < mid && j < right) {
        if (data[i] < data[j])
            temp[k++] = data[i++];
        else
            temp[k++] = data[j++];
    }
    
    while (i < mid)
        temp[k++] = data[i++];
    while (j < right)
        temp[k++] = data[j++];
    
    // Copy sorted elements back to the original array
    for (int i = left; i < right; i++)
        data[i] = temp[i];
}

// Kernel function for parallel merge sort
__global__ void mergeSortKernel(float* data, float* temp, int size, int width) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int left = idx * width * 2;
    
    if (left >= size) return; // Out of bounds
    
    int mid = min(left + width, size);
    int right = min(left + 2 * width, size);
    
    if (mid < right)
        merge(data, temp, left, mid, right);
}

// Host function for iterative parallel merge sort
void mergeSort(float* deviceData, int size) {
    float* deviceTemp;
    cudaMalloc((void**)&deviceTemp, sizeof(float) * size);
    
    for (int width = 1; width < size; width *= 2) {
        int numBlocks = (size / (2 * width) + THREADS - 1) / THREADS;
        mergeSortKernel<<<numBlocks, THREADS>>>(deviceData, deviceTemp, size, width);
        cudaDeviceSynchronize();
    }
    
    cudaFree(deviceTemp);
}

void processFile(const string& inputFile, const string& outputFile) {
    float *hostData, *deviceData;
    int size;

    // CUDA events for timing
    cudaEvent_t start, stop, start_total, stop_total;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventCreate(&start_total);
    cudaEventCreate(&stop_total);
    float milliseconds, total_time;

    /* Importing data and creating host memory */
    ifstream finp(inputFile);
    cudaEventRecord(start);
    
    finp >> size;
    int byteSize = sizeof(float) * size;
    hostData = (float*)malloc(byteSize);
    
    for (int i = 0; i < size; i++) {
        finp >> hostData[i];
    }
    finp.close();

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "[INFO] Time for importing data: " << milliseconds << " ms" << endl;
    cout << "[INFO] Processing File: " << inputFile << ", Array Size: " << size << endl;

    /* Start measuring total time */
    cudaEventRecord(start_total);

    /* Allocating GPU memory */
    cudaEventRecord(start);
    cudaMalloc((void**)&deviceData, byteSize);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "[INFO] Time for allocating GPU memory: " << milliseconds << " ms" << endl;

    /* Copying input memory to the GPU */
    cudaEventRecord(start);
    cudaMemcpy(deviceData, hostData, byteSize, cudaMemcpyHostToDevice);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "[INFO] Time to copy data from CPU to GPU: " << milliseconds << " ms" << endl;

    /* Running Merge Sort on GPU */
    cudaEventRecord(start);
    mergeSort(deviceData, size);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "[INFO] Time for kernel computation on GPU: " << milliseconds << " ms" << endl;

    /* Copying output memory to the CPU */
    cudaEventRecord(start);
    cudaMemcpy(hostData, deviceData, byteSize, cudaMemcpyDeviceToHost);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "[INFO] Time to copy data from GPU to CPU: " << milliseconds << " ms" << endl;

    /* End measuring total time */
    cudaEventRecord(stop_total);
    cudaEventSynchronize(stop_total);
    cudaEventElapsedTime(&total_time, start_total, stop_total);
    cout << "[INFO] Total time: " << total_time << " ms" << endl;

    /* Output results */
    ofstream fout(outputFile);
    fout << size << endl;
    for (int i = 0; i < size; i++) {
        fout << hostData[i] << " ";
    }
    fout.close();
    cout << "[INFO] Output written to " << outputFile << endl;

    /* Freeing GPU Memory */
    cudaFree(deviceData);
    free(hostData);

    /* Destroy CUDA events */
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaEventDestroy(start_total);
    cudaEventDestroy(stop_total);
}

int main() {
    vector<string> inputFiles = {
        "MERGE_SORT_100.INP",
        "MERGE_SORT_500.INP",
        "MERGE_SORT_1000.INP",
        "MERGE_SORT_5000.INP",
        "MERGE_SORT_10000.INP",
        "MERGE_SORT_50000.INP",
        "MERGE_SORT_100000.INP",
        "MERGE_SORT_500000.INP",
        "MERGE_SORT_1000000.INP"
    };

    for (const auto& inputFile : inputFiles) {
        string outputFile = inputFile.substr(0, inputFile.find(".")) + ".OUT";
        processFile(inputFile, outputFile);
    }

    return 0;
}

Overwriting mergeSortCUDA.cu


In [12]:
!nvcc mergeSortCUDA.cu -o mergeSort

In [13]:
!./mergeSort

[INFO] Time for importing data: 0.065728 ms
[INFO] Processing File: MERGE_SORT_100.INP, Array Size: 100
[INFO] Time for allocating GPU memory: 0.178144 ms
[INFO] Time to copy data from CPU to GPU: 0.095968 ms
[INFO] Time for kernel computation on GPU: 0.434688 ms
[INFO] Time to copy data from GPU to CPU: 0.023872 ms
[INFO] Total time: 0.831808 ms
[INFO] Output written to MERGE_SORT_100.OUT
[INFO] Time for importing data: 0.18992 ms
[INFO] Processing File: MERGE_SORT_500.INP, Array Size: 500
[INFO] Time for allocating GPU memory: 0.172416 ms
[INFO] Time to copy data from CPU to GPU: 0.017024 ms
[INFO] Time for kernel computation on GPU: 0.276672 ms
[INFO] Time to copy data from GPU to CPU: 0.019264 ms
[INFO] Total time: 0.56816 ms
[INFO] Output written to MERGE_SORT_500.OUT
[INFO] Time for importing data: 0.176672 ms
[INFO] Processing File: MERGE_SORT_1000.INP, Array Size: 1000
[INFO] Time for allocating GPU memory: 0.147264 ms
[INFO] Time to copy data from CPU to GPU: 0.016512 ms
[INFO

### 4. Cupy Merge Sort

In [17]:
import numpy as np
import cupy as cp
import time
import os

# Define CUDA kernel for merging sorted subarrays
merge_kernel = r'''
extern "C" __global__ void merge(
    float* data, float* temp, int size, int width) {
    
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int left = idx * width * 2;
    
    if (left >= size) return; // Out of bounds
    
    int mid = min(left + width, size);
    int right = min(left + 2 * width, size);
    
    int i = left, j = mid, k = left;
    
    while (i < mid && j < right) {
        if (data[i] < data[j])
            temp[k++] = data[i++];
        else
            temp[k++] = data[j++];
    }
    
    while (i < mid)
        temp[k++] = data[i++];
    while (j < right)
        temp[k++] = data[j++];
    
    for (int i = left; i < right; i++)
        data[i] = temp[i];
}
'''

def merge_sort_gpu(data):
    """ Perform parallel merge sort using CUDA with CuPy. """
    if isinstance(data, np.ndarray):
        data = cp.asarray(data, dtype=cp.float32)

    size = data.size
    temp = cp.zeros_like(data, dtype=cp.float32)

    # Compile CUDA kernel
    kernel = cp.RawKernel(merge_kernel, 'merge')

    # Configure grid & block size
    threads_per_block = 256
    for width in [2**i for i in range(max(1, int(cp.log2(size)) + 1))]:
        num_blocks = (size // (2 * width) + threads_per_block - 1) // threads_per_block
        kernel((num_blocks,), (threads_per_block,), (data, temp, size, width))
        cp.cuda.stream.get_current_stream().synchronize()

    del temp  # Free temporary GPU memory
    cp.get_default_memory_pool().free_all_blocks()  # Force GPU memory cleanup

    return data

if __name__ == "__main__":
    input_files = [
        "MERGE_SORT_100.INP",
        "MERGE_SORT_500.INP",
        "MERGE_SORT_1000.INP",
        "MERGE_SORT_5000.INP",
        "MERGE_SORT_10000.INP",
        "MERGE_SORT_50000.INP",
        "MERGE_SORT_100000.INP",
        "MERGE_SORT_500000.INP",
        "MERGE_SORT_1000000.INP"
    ]

    for input_file in input_files:
        output_file = input_file.replace(".INP", ".OUT")

        if not os.path.exists(input_file):
            print(f"Skipping {input_file} (File not found)")
            continue

        print(f"\nProcessing {input_file}...")

        # Load data
        with open(input_file, "r") as f:
            size = int(f.readline().strip())
            array = np.array(list(map(float, f.readline().split())), dtype=np.float32)

        print(f"Array Size: {size}")

        # Start total timing
        start_total = time.time()

        # Copy to GPU
        start = time.time()
        data_gpu = cp.asarray(array)
        cp.cuda.stream.get_current_stream().synchronize()
        end = time.time()
        print(f"Time to copy data to GPU: {(end - start) * 1000:.4f} ms")

        # Sort the array
        start = time.time()
        sorted_gpu = merge_sort_gpu(data_gpu)
        cp.cuda.stream.get_current_stream().synchronize()
        end = time.time()
        print(f"Time for kernel computation on GPU: {(end - start) * 1000:.4f} ms")

        # Copy back to CPU
        start = time.time()
        sorted_array = cp.asnumpy(sorted_gpu)
        cp.cuda.stream.get_current_stream().synchronize()
        end = time.time()
        print(f"Time to copy data from GPU: {(end - start) * 1000:.4f} ms")

        # Free GPU memory
        del data_gpu, sorted_gpu
        cp.get_default_memory_pool().free_all_blocks()

        end_total = time.time()
        print(f"Total time: {(end_total - start_total) * 1000:.4f} ms")

        # Save output
        with open(output_file, "w") as f:
            f.write(f"{size}\n")
            f.write(" ".join(map(str, sorted_array)) + "\n")

        print(f"Output saved to {output_file}")

    print("\nAll input files processed successfully!")


Processing MERGE_SORT_100.INP...
Array Size: 100
Time to copy data to GPU: 1.2681 ms
Time for kernel computation on GPU: 0.9990 ms
Time to copy data from GPU: 0.3119 ms
Total time: 2.7633 ms
Output saved to MERGE_SORT_100.OUT

Processing MERGE_SORT_500.INP...
Array Size: 500
Time to copy data to GPU: 0.5169 ms
Time for kernel computation on GPU: 1.0090 ms
Time to copy data from GPU: 0.0579 ms
Total time: 1.9238 ms
Output saved to MERGE_SORT_500.OUT

Processing MERGE_SORT_1000.INP...
Array Size: 1000
Time to copy data to GPU: 0.3240 ms
Time for kernel computation on GPU: 1.3871 ms
Time to copy data from GPU: 0.2453 ms
Total time: 2.1415 ms
Output saved to MERGE_SORT_1000.OUT

Processing MERGE_SORT_5000.INP...
Array Size: 5000
Time to copy data to GPU: 0.3104 ms
Time for kernel computation on GPU: 4.8506 ms
Time to copy data from GPU: 0.3037 ms
Total time: 5.7735 ms
Output saved to MERGE_SORT_5000.OUT

Processing MERGE_SORT_10000.INP...
Array Size: 10000
Time to copy data to GPU: 0.2785

### 5. RAPIDS Merge Sort Cupy kernel

In [16]:
import cupy as cp
import cudf
import time
import os

# CUDA Kernel for Merge Sort
merge_sort_kernel = cp.RawKernel(r'''
extern "C" __global__ void merge_sort(int* arr, int* temp, int n, int width) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int left = tid * 2 * width;
    int mid = min(left + width, n);
    int right = min(left + 2 * width, n);

    if (mid >= right) return;

    int i = left, j = mid, k = left;
    while (i < mid && j < right) {
        if (arr[i] <= arr[j]) {
            temp[k++] = arr[i++];
        } else {
            temp[k++] = arr[j++];
        }
    }
    while (i < mid) temp[k++] = arr[i++];
    while (j < right) temp[k++] = arr[j++];

    for (int l = left; l < right; l++) {
        arr[l] = temp[l];
    }
}
''', 'merge_sort')

# Custom GPU Merge Sort Function
def gpu_merge_sort(self):
    if self.shape[1] != 1:
        raise ValueError("Merge sort only supports single-column cuDF DataFrames")

    total_start = time.time()

    # 1. Memory Allocation (GPU) - Fixed to 1D
    start_alloc = time.time()
    arr_gpu = cp.empty(self.shape[0], dtype=cp.int32)  # ✅ Ensure it's 1D
    temp_gpu = cp.empty(self.shape[0], dtype=cp.int32)
    end_alloc = time.time()

    # 2. Copy Data from CPU to GPU
    start_copy_h2d = time.time()
    arr_gpu[:] = cp.asarray(self.iloc[:, 0].to_cupy(), dtype=cp.int32)  # ✅ Correct extraction
    end_copy_h2d = time.time()

    # 3. Kernel Execution (Iterative Merge Sort)
    start_kernel = time.time()
    n = arr_gpu.size
    threads_per_block = 256
    blocks_per_grid = (n + threads_per_block - 1) // threads_per_block

    width = 1
    while width < n:
        grid_size = (n // (2 * width) + (1 if n % (2 * width) != 0 else 0))  # Prevent excessive blocks
        merge_sort_kernel((grid_size,), (threads_per_block,), (arr_gpu, temp_gpu, n, width))
        cp.cuda.Device(0).synchronize()
        width *= 2
    end_kernel = time.time()

    # 4. Copy Data from GPU to CPU
    start_copy_d2h = time.time()
    sorted_result = arr_gpu.get()
    end_copy_d2h = time.time()

    # Total execution time
    total_end = time.time()

    # Convert back to cuDF DataFrame
    sorted_df = cudf.DataFrame(sorted_result, columns=self.columns)

    # Convert all times to milliseconds
    alloc_time = (end_alloc - start_alloc) * 1000
    copy_h2d_time = (end_copy_h2d - start_copy_h2d) * 1000
    kernel_time = (end_kernel - start_kernel) * 1000
    copy_d2h_time = (end_copy_d2h - start_copy_d2h) * 1000
    total_time = (total_end - total_start) * 1000

    # Print Timing Information
    print(f"Memory Allocation Time: {alloc_time:.4f} ms")
    print(f"CPU → GPU Copy Time: {copy_h2d_time:.4f} ms")
    print(f"Kernel Execution Time: {kernel_time:.4f} ms")
    print(f"GPU → CPU Copy Time: {copy_d2h_time:.4f} ms")
    print(f"Total Execution Time: {total_time:.4f} ms")

    return sorted_df

# Monkey-patch cuDF DataFrame to add the custom method
cudf.DataFrame.gpu_merge_sort = gpu_merge_sort

# List of input files
input_files = [
    "MERGE_SORT_100.INP",
    "MERGE_SORT_500.INP",
    "MERGE_SORT_1000.INP",
    "MERGE_SORT_5000.INP",
    "MERGE_SORT_10000.INP",
    "MERGE_SORT_50000.INP",
    "MERGE_SORT_100000.INP",
    "MERGE_SORT_500000.INP",
    "MERGE_SORT_1000000.INP"
]

if __name__ == "__main__":
    for input_file in input_files:
        output_file = input_file.replace(".INP", ".OUT")

        if not os.path.exists(input_file):
            print(f"Skipping {input_file} (File not found)")
            continue

        print(f"\nProcessing {input_file}...")

        # Load data
        with open(input_file, "r") as f:
            size = int(f.readline().strip())
            array = list(map(int, f.readline().split()))

        print(f"Array Size: {size}")

        # Convert to cuDF DataFrame
        df = cudf.DataFrame({'values': array})

        # Sort using GPU Merge Sort
        sorted_df = df.gpu_merge_sort()

        # Save output
        sorted_array = sorted_df['values'].to_pandas().to_numpy()

        with open(output_file, "w") as f:
            f.write(f"{size}\n")
            f.write(" ".join(map(str, sorted_array)) + "\n")

        print(f"Output saved to {output_file}")

    print("\nAll input files processed successfully!")


Processing MERGE_SORT_100.INP...
Array Size: 100
Memory Allocation Time: 0.2778 ms
CPU → GPU Copy Time: 1.1435 ms
Kernel Execution Time: 1.2572 ms
GPU → CPU Copy Time: 0.0675 ms
Total Execution Time: 2.7473 ms
Output saved to MERGE_SORT_100.OUT

Processing MERGE_SORT_500.INP...
Array Size: 500
Memory Allocation Time: 0.2205 ms
CPU → GPU Copy Time: 0.9499 ms
Kernel Execution Time: 0.4997 ms
GPU → CPU Copy Time: 0.0472 ms
Total Execution Time: 1.7183 ms
Output saved to MERGE_SORT_500.OUT

Processing MERGE_SORT_1000.INP...
Array Size: 1000
Memory Allocation Time: 0.1273 ms
CPU → GPU Copy Time: 0.8874 ms
Kernel Execution Time: 0.7994 ms
GPU → CPU Copy Time: 0.0455 ms
Total Execution Time: 1.8604 ms
Output saved to MERGE_SORT_1000.OUT

Processing MERGE_SORT_5000.INP...
Array Size: 5000
Memory Allocation Time: 0.1366 ms
CPU → GPU Copy Time: 0.8128 ms
Kernel Execution Time: 4.1521 ms
GPU → CPU Copy Time: 0.0620 ms
Total Execution Time: 5.1641 ms
Output saved to MERGE_SORT_5000.OUT

Processin